In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [3]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [6]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
['addiction', 'affirmation_distraction_addiction', 'anxiety', 'challenge_thought_anxiety', 'depression', 'explain_distraction_addiction', 'explainreccdist_depression', 'first_substance_addiction', 'go_plan_for_things', 'goodbye', 'greeting', 'how_wreck', 'ideas_wrecking', 'need_help', 'need_help_adiction', 'need_ideas_addiction', 'no_distraction_addiction', 'no_distraction_addiction_after', 'plan_for_thing', 'ques_to_q1_anxiety', 'question1_anxiety', 'question1_depression', 'question_to_question1_depression', 'question_to_question2_anxiety', 'question_to_question2_depression', 'question_to_question3_depression', 'questtion_to_question4_depression', 'stating_problem', 'thanks', 'the_end', 'the_end_bad', 'things_addiction', 'what_to_do_addiction2', 'work_on_it', 'wo

In [7]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from D:\COLLEGE\BE Project\github_CB\responses\model.tflearn


In [8]:
yeah_dict = {'problem_got': ('response_to_condolence', 'How would you categorize this issue 1.Angry, 2.Anxiety, 3.Depression'), 'depressed':'response_to_condolence_depression', 'write_thought':'affirmation_to_ques1_depression', 'catastrophe':'affirmation_to_question2_depression', 'write_thought_anxiety':'affirmation_to_ques1_anxiety', 'future_assuming':'affirmation_ques2_anxiety', 'distortion':'affirmation_to_question3_depression', 'emotionalreasoning':'afftoques4_depression', 'write_thought_anxiety':'affirmation_to_ques1_anxiety', 'future_assuming':'affirmation_ques2_anxiety'}
no_dict = {'write_thought':'negation_to_ques1_depression', 'catastrophe':'negation_to_question2_depression','distortion':'negation_to_question4_depression', 'rewritethoughts':'negationtoques4_depression', 'write_thought_anxiety':'negation_to_ques1_anxiety', 'future_assuming':'affirmation_negation_ques2_anxiety', 'emotionalreasoning':'negationtoques4_depression'}
okay_dict = {'depressed':'response_to_condolence_depression','problem_got':'response_to_condolence','questoques1explained':'REquestion_to_question1_depression', 'questoques1recognized':'question2_depression', 'ques2explained':'requestion_to_question2_depression','makingUnderstandques2':'question3_depression','ques3explained':'got_it_depression', 'everythingExplained':'repquestion_to_question3_depression', 'gotchaques3':'question4_depression', 'ques4explained':'affToQuestoques4_depression', 'questoques1explained_anxiety':'reques_to_ques1_anxiety', 'questoques1confirmed_anxiety':'question2_anxiety', 'future_assuming_explanation':'reques_to_ques2_anxiety', 'affirmation_anxiety':'okay_ques2_anxiety'}

In [11]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=True):
    results = classify(sentence)
    for i in intents['intents']:
                # find a tag matching the first result
                if sentence in ['yes','yeah']:
#                     print('in yeah loop')
                    tag = yeah_dict[context[userID]]
                    if i['tag'] == tag:
                        resp = random.choice(i['responses'])
                        if 'context_set' in i:
                            context[userID] = i['context_set']
#                         print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                        return('first yeah  '+str(resp))
                elif sentence in ['no','nope', 'not at all']:
#                     print('in no loop')
                    tag = no_dict[context[userID]]
                    if i['tag'] == tag:
                        resp = random.choice(i['responses'])
                        if 'context_set' in i:
                            context[userID] = i['context_set']
#                         print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                        return('first  '+str(resp))
                elif sentence in ['okay','cool', 'gotcha', 'it is', 'got it', 'alright', 'it does']:
#                     print('in okay loop')
                    tag = okay_dict[context[userID]]
                    if i['tag'] == tag:
                        resp = random.choice(i['responses'])
                        if 'context_set' in i:
                            context[userID] = i['context_set']
#                         print('new context  ', context[userID])
                    #if 'context_filter' in i:
                        #print('context set in first if : ',context[userID])
                        #if context[userID]==i['context_filter']:
                            #print('response in second if :', i['responses'])
                            #resp = random.choice(i['responses'])
                        #if 'context_set' in i:
                            #if show_details: print ('context in third if :', i['context_set'])
                            #context[userID] = i['context_set']

                        return('first  '+str(resp))
    # if we have a classification then find the matching intent tag
    if results:
#         print(results)
        # loop as long as there are matches to process
        for i in results:
#             print(i)
            for j in intents['intents']:
                
                if i[0]==j['tag']:
#                     try:
# #                         print('past context  ', context[userID])
#                     except:
#                         pass
                    if not 'context_filter' in j or \
                        (userID in context and 'context_filter' in j and j['context_filter'] == context[userID]):
#                         print('in second if')
                        if show_details: print ('in first if tag:', j['tag'])
#                         resp = random.choice(j['responses'])
                        if 'context_set' in j: 
                            context[userID] = j['context_set']
#                             print('new context  ', context[userID])
                        return(resp)
                

        

In [ ]:
%mkdir templates -p

In [ ]:
%%writefile templates/index.html

<!DOCTYPE html>
<html>
  <title>Restaurant Chatbot</title>
  <head>
    <link href='https://fonts.googleapis.com/css?family=Alegreya' rel='stylesheet'>
    <link rel="icon" href="">
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
    <style>
      body {
        font-family: monospace;
        background-position: center;
        background-repeat: no-repeat;
        background-size: cover;
        background-attachment: fixed;
      }
      h1 {
        border: 1px solid white;
        border-radius: 5px;
        color: #ce9e62;
        display: inline-block;Alegreya
        font-size: 3em;
        margin: 5;
        padding: 10px;
      }
      #chatbox {
        margin-top: auto;
        margin-bottom: auto;
        margin-left: auto;
        margin-right: auto;
        width: 40%;
        margin-top: 60px;
      }
      #userInput {
        margin-left: auto;
        margin-right: auto;
        width: 40%;
        margin-top: 60px;
      }
      #textInput {
        width: 90%;
        border: none;
        border-bottom: 3px solid black;
        font-family: 'Alegreya';
        font-size: 17px;
      }
      .userText {
        color: white;
        background-color: #4b6777; 
        font-family: 'Alegreya';
        font-size: 17px;
        text-align: center;
        line-height: 30px;
        border-radius: 5px;
      }
      .userText span {
        padding:10px;
        border-radius: 5px;
      }
      .botText {
        color: #ce9e62;
        background-color: #2c2c2c;
        font-family: 'Alegreya';
        font-size: 17px;
        text-align: center;
        line-height: 30px;
        border-radius: 5px;
      }
      .botText span {
        padding: 10px;
        border-radius: 5px;
      }
      .boxed {
        margin-left: auto;
        margin-right: auto;
        width: 100%;
        margin-top: 60px;
        border-radius: 5px;
      }
      input[type=text] {
        bottom: 0;
        width: 40%;
        padding: 12px 20px;
        margin: 8px 0;
        box-sizing: border-box;
        position: fixed;
        border-radius: 5px;
      }
    </style>
  </head>
  <body background="{{ url_for('static', filename='images/slider.jpg') }}">
    <img />
    <center>
      <h1>
      Restaurant Chatbot
      </h1>
    </center>
    <div class="boxed">
      <div>
        <div id="chatbox">
        </div>
      </div>
        <div id="userInput">
          <input id="nameInput" type="text" name="msg" placeholder="Ask me anything..." />
        </div>
<script>
        function getBotResponse() {
          var rawText = $("#nameInput").val();
          var userHtml = '<p class="userText"><span>' + rawText + "</span></p>";
          $("#nameInput").val("");
          $("#chatbox").append(userHtml);
          document
            .getElementById("userInput")
            .scrollIntoView({ block: "start", behavior: "smooth" });
          $.get("/get", { msg: rawText }).done(function(data) {
            var botHtml = '<p class="botText"><span>' + data + "</span></p>";
            $("#chatbox").append(botHtml);
            document
              .getElementById("userInput")
              .scrollIntoView({ block: "start", behavior: "smooth" });
          });
        }
        $("#nameInput").keypress(function(e) {
          if (e.which == 13) {
            getBotResponse();
          }
        });
</script>
</div>
</body>
</html>

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route('/')
def index():
  return render_template('index.html')


@app.route('/get')
def bot_response():
  user_text = request.args.get("msg")
  return str(response(user_text))
  
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bd979e46e386.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Feb/2021 18:56:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Feb/2021 18:56:06] "GET /static/images/slider.jpg HTTP/1.1" 404 -
[2021-02-14 18:56:08,387] ERROR in app: Exception on /get [GET]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    re

in first if tag: greeting


else:
                    print('in else loop')
                    if 'context_filter' in j and j['context_filter'] == context[userID]:
                        resp = random.choice(j['responses'])
                        print('tag: ', j['tag'])
                        if 'context_set' in j: 
                            context[userID] = j['context_set']
                            print('new context  ', context[userID])
                        return(resp)

while results:
            for i in intents['intents']:
               
                    if i['tag'] == results[0][0]:
                        print('past context ',context, ' tag:', i['tag'])

                        if not 'context_filter' in i or \
                            (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                            if show_details: print ('in first if tag:', i['tag'])
                            resp = random.choice(i['responses'])

                        if 'context_filter' in i:
                            if show_details: print('context_filter in second if : ', i['context_filter'])
                            resp = random.choice(i['responses'])

                            # set context for this intent if necessary
                        if 'context_set' in i:
                            if show_details: print ('context in third if : ', i['context_set'])
                            context[userID] = i['context_set']
                            print('new context ', context)

                        if 'context_filter' in i:
                            if show_details: print('context_filter in fourth if : ', i['context_filter'])


                        return print('second  ' ,resp)


In [ ]:
response('hi')

In [ ]:
response('help me feel better')

In [ ]:
response('im having problem with')

In [ ]:
response('it is')

In [ ]:
response('depression')

In [ ]:
response('it is')

In [ ]:
response('work on it')

In [ ]:
response('challenge my thought')

In [ ]:
response('i have this problem')

In [ ]:
response('not at all')

In [ ]:
response('catastrophe')

In [ ]:
response('it is')

In [ ]:
response('yes')

In [ ]:
response('got it')

In [ ]:
response('no')

In [ ]:
response('no')

In [ ]:
response('hi')

In [ ]:
response('anxiety')

In [ ]:
response('work on it')

In [ ]:
response('challenge my thought')

In [ ]:
response('i have this problem')

In [ ]:
response('what is that?')

In [ ]:
response('got it')

In [ ]:
response('yes')

In [ ]:
response('okay')

In [ ]:
response('no')

In [ ]:
response('no')